<a href="https://colab.research.google.com/github/nastyaskutina/fairytales/blob/main/data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
!pip install stanza
import stanza
stanza.download('ru')
nlp = stanza.Pipeline('ru')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 4.1 MB/s 
     |████████████████████████████████| 240 kB 45.5 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234927 sha256=bf3d8b6fb120b0511b548d4ca4eda5c5706baa90054aee1a4ee3d92779adad33
  Stored in directory: /root/.cache/pip/wheels/f3/e3/f2/1de1c2e3ed742e1df73e0f15d58864e50c7e64f607b548d6cf
Successfully built emoji


INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [3]:
def make_coref_table1(df1):
  big_dict = {'id1': [], 'id2': [],
              'num1': [], 'num2': [], 'gen1': [], 'gen2': [], 
              'pers1': [],'pers2': [], 'anim1': [], 'anim2': []}
  list_id = list(df1[df1['ids'] != '-'].ids)
  for id1 in list_id:
    for id2 in list_id[list_id.index(id1)+1:]:
      big_dict['id1'].append(list(df1[df1['ids'] == id1].ids)[0])
      big_dict['id2'].append(list(df1[df1['ids'] == id2].ids)[0])
      big_dict['num1'].append(list(df1[df1['ids'] == id1].Number)[0])
      big_dict['num2'].append(list(df1[df1['ids'] == id2].Number)[0])
      big_dict['gen1'].append(list(df1[df1['ids'] == id1].Gender)[0])
      big_dict['gen2'].append(list(df1[df1['ids'] == id2].Gender)[0])
      big_dict['pers1'].append(list(df1[df1['ids'] == id1].Person)[0])
      big_dict['pers2'].append(list(df1[df1['ids'] == id2].Person)[0])
      big_dict['anim1'].append(list(df1[df1['ids'] == id1].Animacy)[0])
      big_dict['anim2'].append(list(df1[df1['ids'] == id2].Animacy)[0])
  return big_dict

In [4]:
def add_inf1(doc, df1):
  #lst = list(df1[3])
  # event chain for sentence
  event_chain = []
  details = []
  feats = []
  #counter = 0
  part_idx = 0
  ids = []
  word_forms = {'text': [], 'lemmas': []}
  for sentence in doc.sentences:
    all_verbs = []
    verb_ids = []
    events = {}
    all_participants = []
    for word in sentence.words:
      part_idx += 1
      word_forms['text'].append(word.text)
      word_forms['lemmas'].append(word.lemma)
      part_idx, ids, details, feats = for_good_word(word, part_idx, ids, details, feats)

      if word.pos == 'VERB':
        all_verbs.append(word)
        verb_ids.append(word.id)
        events[word.lemma] = []
      if word.deprel == 'obj' or word.deprel == 'nsubj':
        all_participants.append((word, part_idx))
    
    for participant, part_idx in all_participants:
      if participant.head in verb_ids:
        idx = verb_ids.index(participant.head)
        events[all_verbs[idx].lemma].append((participant.lemma, participant.deprel, part_idx))
    for key in events:
      if len(events[key]) != 0:
        event = (key, tuple(events[key]))
        event_chain.append(event)

  df1.loc[:, 'text'] = word_forms['text']
  df1.loc[:, 'lemma'] = word_forms['lemmas']
  df1.loc[:, 'details'] = details
  df1.loc[:, 'ids'] = ids
  df1.loc[:, 'Number'] = [f[0] for f in feats]
  df1.loc[:, 'Gender'] = [f[1] for f in feats]
  df1.loc[:, 'Person'] = [f[2] for f in feats]
  df1.loc[:, 'Animacy'] = [f[3] for f in feats]

  return df1, event_chain

In [5]:
def for_good_word(word, part_idx, ids, details, feats):
  if word.pos == 'VERB':
    ids.append('-')
    #all_verbs.append(word)
    #verb_ids.append(word.id)
    #events[word.lemma] = []
    details.append('VERB')
    feats.append(('-', '-', '-', '-'))
    
  elif word.deprel == 'obj' or word.deprel == 'nsubj':
    ids.append(part_idx)
    #all_participants.append(word)
    start_feats = ['-', '-', '-', '-']
    if word.feats:
      word_feats = word.feats.split('|')
      for feat in word_feats:
        if feat.split('=')[0] == 'Number':
          start_feats[0] = feat.split('=')[1]
        elif feat.split('=')[0] == 'Gender':
          start_feats[1] = feat.split('=')[1]
        elif feat.split('=')[0] == 'Person':
          start_feats[2] = feat.split('=')[1]
        elif feat.split('=')[0] == 'Animacy':
          start_feats[3] = feat.split('=')[1]
    feats.append(tuple(start_feats))
    if word.deprel == 'obj':
      details.append('obj')
    elif word.deprel == 'nsubj':
      details.append('nsubj')
      
  else:
    details.append('-')
    feats.append(('-', '-', '-', '-'))
    ids.append('-')
  return part_idx, ids, details, feats

In [9]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 7.4 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=d0ba4207e8f4e99c84d009ae3e9d09cab8fb91910bba5e515f194e32fcf3dd83
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [10]:
def coref_check_a(id1, id2, antecendent, ft_table, corefs):
  counter = 0
  for feat in ['Number', 'Gender', 'Animacy']:
    word_feat1 = list(ft_table[ft_table['ids'] == id1][feat])[0]
    word_feat2 = list(ft_table[ft_table['ids'] == id2][feat])[0]
    word_feat3 = list(ft_table[ft_table['ids'] == antecendent][feat])[0]
    if word_feat1 != word_feat2 and word_feat1 != '-' and word_feat2 != '-':
      counter += 1
    if word_feat1 != word_feat3 and word_feat1 != '-' and word_feat3 != '-':
      counter += 1
  if counter < 1:
    corefs[id1] = antecendent


def coref_check(id1, id2, ft_table, corefs):
  counter = 0
  for feat in ['Number', 'Gender', 'Animacy']:
    word_feat1 = list(ft_table[ft_table['ids'] == id1][feat])[0]
    word_feat2 = list(ft_table[ft_table['ids'] == id2][feat])[0]
    if word_feat1 != word_feat2 and word_feat1 != '-' and word_feat2 != '-':
      counter += 1
  if counter < 1:
    corefs[id1] = id2

In [11]:
def coref_check_n(id1, id2, ft_table):
  counter = 0
  for feat in ['Number', 'Gender', 'Animacy']:
    word_feat1 = list(ft_table[ft_table['ids'] == id1][feat])[0]
    word_feat2 = list(ft_table[ft_table['ids'] == id2][feat])[0]
    if word_feat1 != word_feat2 and word_feat1 != '-' and word_feat2 != '-':
      counter += 1
  result = counter < 1
  return result

In [33]:
def coref_change(id1, id2, corefs):
  for k in corefs:
    if id1 == corefs[k]:
      corefs[k] = id2

In [12]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [13]:
from collections import Counter

In [16]:
def for_gen(c, ft):
  feats = []
  for item in c[1]:
    if item == '-':
      feats.append(0)
    elif item == 'Masc':
      feats.append(1)
    elif item == 'Fem':
      feats.append(2)
    elif item == 'Neut':
      feats.append(3)
      
  if c[0] == 'gen1':
    ft.gen1 = feats
  elif c[0] == 'gen2':
    ft.gen2 = feats

In [17]:
def for_num(c, ft):
  feats = []
  for item in c[1]:
    if item == '-':
      feats.append(0)
    elif item == 'Sing':
      feats.append(1)
    elif item == 'Plur':
      feats.append(2)
      
  if c[0] == 'num1':
    ft.num1 = feats
  elif c[0] == 'num2':
    ft.num2 = feats

In [18]:
def for_pers(c, ft):
  feats = []
  for item in c[1]:
    if item == '-':
      feats.append(0)
    elif item == '1':
      feats.append(1)
    elif item == '2':
      feats.append(2)
    elif item == '3':
      feats.append(3)
  
  if c[0] == 'pers1':
    ft.pers1 = feats
  elif c[0] == 'pers2':
    ft.pers2 = feats

In [19]:
def for_anim(c, ft):
  feats = []
  for item in c[1]:
    if item == '-':
      feats.append(0)
    elif item == 'Inan':
      feats.append(1)
    elif item == 'Anim':
      feats.append(2)
  
  if c[0] == 'anim1':
    ft.anim1 = feats
  elif c[0] == 'anim2':
    ft.anim2 = feats

In [20]:
def table_fix(ft):
  cols = [('gen1', ft.gen1), ('gen2', ft.gen2), ('num1', ft.num1), ('num2', ft.num2), 
          ('pers1', ft.pers1), ('pers2', ft.pers2), ('anim1', ft.anim1), ('anim2', ft.anim2)]
  for c in cols:
    if c[0].startswith('gen'):
      for_gen(c, ft)
    elif c[0].startswith('num'):
      for_num(c, ft)
    elif c[0].startswith('pers'):
      for_pers(c, ft)
    elif c[0].startswith('anim'):
      for_anim(c, ft)


In [21]:
def text_preproc(text):
  table = pd.DataFrame()
  ft_table, ev_chain = add_inf1(nlp(text), table)
  ft_dic = make_coref_table1(ft_table)
  ft = pd.DataFrame(ft_dic)
  table_fix(ft)
  columns=['id1', 'id2']
  ft_X = ft.drop(columns, axis = 1)
  ft_y_pred = coref_model.predict(ft_X)
  ft['pred_coref'] = list(ft_y_pred)
  return ft, ev_chain, ft_table

In [22]:
def get_coref_chains(ft, ev_chain, ft_table):
  ft_cor = {}
  for i, row in ft[['id1', 'id2']][ft['pred_coref'] == 1].iterrows():
    if row['id1'] not in ft_cor:
      ft_cor[row['id1']] = []
    ft_cor[row['id1']].append(row['id2'])

    if row['id2'] not in ft_cor:
      ft_cor[row['id2']] = []
    ft_cor[row['id2']].append(row['id1']) 
  

  coref_chains = {}
  all_events = []
  corefs = {}
  for event in ev_chain:
    for part in event[1]:
      all_events.append((event[0], part[1]))
      chain_counter = 0
      if part[2] in ft_cor:
        if part[2] in corefs:
          antecendent = corefs[part[2]]
          coref_chains[antecendent]['names'].append(part[0])
          coref_chains[antecendent]['events'].append((event[0], part[1], part[0]))
          chain_counter = 1
          for idx in [idx for idx in ft_cor[part[2]] if idx > part[2]]:
            coref_check_a(idx, part[2], antecendent, ft_table, corefs)
        else:
          for idx in [idx for idx in ft_cor[part[2]] if idx > part[2]]:
            coref_check(idx, part[2], ft_table, corefs)
      
      if chain_counter == 0:
        for key in coref_chains:
          if part[0] in coref_chains[key]['names']:
              # проверка
            if coref_check_n(key, part[2], ft_table):
              coref_chains[key]['names'].append(part[0])
              coref_chains[key]['events'].append((event[0], part[1], part[0]))
              coref_change(part[2], key, corefs)
              chain_counter = 1
              break
            else:
              continue
        
      if chain_counter == 0:
        coref_chains[part[2]] = {}
        coref_chains[part[2]]['names'] = [part[0]]
        coref_chains[part[2]]['events'] = [(event[0], part[1], part[0])]
  
  return coref_chains, all_events

In [23]:
def add_data(coref_chains, coref_event_count, coref_role_count):
  for cor in coref_chains:
    events = coref_chains[cor]['events']
    names = coref_chains[cor]['names']
    name = None
    for item in Counter(names).most_common():
      for p in morph.parse(item[0]):
        if 'NOUN' in p.tag:
          name = item[0]
          break
      if name != None:
        break
    
    if len(events) > 1:
      for ev1 in events[:-1]:
        for ev2 in events[events.index(ev1)+1:]:
          coref_event_count.append((ev1[:-1], ev2[:-1]))
          if name != None:
            coref_role_count.append((ev1[:-1], ev2[:-1], name))

In [24]:
import os

In [28]:
df_cat = pd.read_csv('coref_inf.tsv', sep='\t')  
df_cat = df_cat.drop(['Unnamed: 0'], axis =1 )
table_fix(df_cat)

In [30]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split


columns=['coref']

X = df_cat.drop(columns, axis = 1)
y = df_cat['coref']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [31]:
lg = LogisticRegression()
coref_model = lg.fit(X_train, y_train)
y_pred = coref_model.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" 
      % (X_test.shape[0], (y_test != y_pred).sum()))
print((y_test != y_pred).sum() / X_test.shape[0] * 100)

Number of mislabeled points out of a total 85268 points : 22978
26.947975793967256


In [36]:
f_texts = []
dir_name = 'data'
for filename in os.listdir(dir_name):
  #print(filename)
  with open(os.path.join(dir_name, filename), 'r', encoding='utf-8') as f:
    text = f.read()
    f_texts.append(text)

In [37]:
coref_event_count = []
coref_role_count = []
all_events = []
event_counter = 0
for t in f_texts:
  ft, ev_chain, ft_table = text_preproc(t)
  try:
    coref_chains, text_events = get_coref_chains(ft, ev_chain, ft_table)
  except LookupError:
    print('sorry')
    continue
  event_counter += len(ev_chain)
  all_events.extend(text_events)
  add_data(coref_chains, coref_event_count, coref_role_count)
  print('ready')

ready


In [38]:
with open('coref_event_count.txt', 'a', encoding='utf-8') as f:
  for pair in coref_event_count:
    f.write(pair[0][0]+'\t'+pair[0][1]+'\t'+pair[1][0]+'\t'+pair[1][1]+'\n')

In [39]:
with open('coref_role_count.txt', 'a', encoding='utf-8') as f:
  for pair in coref_role_count:
    f.write(pair[0][0]+'\t'+pair[0][1]+'\t'+pair[1][0]+'\t'+pair[1][1]+'\t'+pair[2]+'\n')

In [40]:
with open('all_events.txt', 'a', encoding='utf-8') as f:
  for pair in all_events:
    f.write(pair[0]+'\t'+pair[1]+'\n')

In [41]:
with open('event_count.txt', 'a', encoding='utf-8') as f:
  f.write('\n'+str(event_counter))